In [ ]:
!kaggle datasets download -d hugodarwood/epirecipes

In [11]:
!unzip -q epirecipes.zip

In [12]:
# 전체 데이터셋 로드
import json

with open("./full_format_recipes.json") as json_data:
    recipe_data = json.load(json_data)

In [13]:
# 데이터셋 필터링
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

In [14]:
filtered_data

['Recipe for Lentil, Apple, and Turkey Wrap  | 1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool. 2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper. 3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.',
 'Recipe for Boudin Blanc Terrine with Red Onion Confit  | Combine first 9 ingredients in heavy medium 

In [17]:
# 레시피 개수 확인
n_recipes = len(filtered_data)
print(f"recipes for {n_recipes}개")

recipes for 20111개


In [26]:
# tokenization

import re
import string

# 구두점을 분리하여 '단어'로 토크나이징
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s

text_data = [pad_punctuation(x) for x in filtered_data]
print(text_data[0])
print(len(text_data[0]))

Recipe for Lentil , Apple , and Turkey Wrap | 1 . Place the stock , lentils , celery , carrot , thyme , and salt in a medium saucepan and bring to a boil . Reduce heat to low and simmer until the lentils are tender , about 30 minutes , depending on the lentils . ( If they begin to dry out , add water as needed . ) Remove and discard the thyme . Drain and transfer the mixture to a bowl ; let cool . 2 . Fold in the tomato , apple , lemon juice , and olive oil . Season with the pepper . 3 . To assemble a wrap , place 1 lavash sheet on a clean work surface . Spread some of the lentil mixture on the end nearest you , leaving a 1 - inch border . Top with several slices of turkey , then some of the lettuce . Roll up the lavash , slice crosswise , and serve . If using tortillas , spread the lentils in the center , top with the turkey and lettuce , and fold up the bottom , left side , and right side before rolling away from you . 
935


In [27]:
import tensorflow as tf

# 텐서플로 데이터셋으로 변환하기
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(32)
    .shuffle(1000)
)

from tensorflow.keras import layers

# Textvectorization layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens= 10000,
    output_mode="int",
    output_sequence_length= 200 + 1,)# 훈련 세트에 층 적용

vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-04-15 15:41:16.429574: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-15 15:41:16.430342: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-04-15 15:41:16.519491: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-04-15 15:41:16.523625: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [33]:
len(vocab)
vocab

['',
 '[UNK]',
 '.',
 ',',
 'and',
 'to',
 'in',
 'the',
 'with',
 'a',
 'until',
 '1',
 'minutes',
 '-',
 'of',
 '2',
 'for',
 'heat',
 'add',
 'about',
 'over',
 'bowl',
 ';',
 '/',
 'salt',
 'into',
 'recipe',
 '|',
 'on',
 'medium',
 'large',
 'mixture',
 '4',
 'pepper',
 '(',
 ')',
 '3',
 'oil',
 'is',
 'water',
 'transfer',
 'or',
 'stir',
 'cook',
 'pan',
 'remaining',
 'then',
 'oven',
 'stirring',
 'cover',
 'butter',
 'from',
 'cup',
 'inch',
 'sauce',
 'sugar',
 'skillet',
 'at',
 'baking',
 '5',
 'cool',
 'it',
 'be',
 'season',
 'place',
 'small',
 'each',
 'let',
 'serve',
 'boil',
 'simmer',
 'remove',
 'top',
 'whisk',
 'cut',
 'high',
 'cream',
 'ahead',
 'heavy',
 'are',
 'saucepan',
 'garlic',
 '10',
 'chicken',
 'bring',
 'tender',
 'preheat',
 'using',
 'sprinkle',
 'tablespoons',
 'occasionally',
 'brown',
 'if',
 'just',
 'dough',
 'can',
 'side',
 'bake',
 'spoon',
 'up',
 'golden',
 'pot',
 'through',
 'combine',
 'juice',
 'sheet',
 'hours',
 'pour',
 'chill',

In [34]:
# 토큰:단어 매핑 샘플 출력하기
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: ,
4: and
5: to
6: in
7: the
8: with
9: a


In [36]:
# 동일 샘플을 정수로 변환하여 출력하기
example_tokenised = vectorize_layer(text_data[0])
print(example_tokenised.numpy())

[  26   16 1733    3  428    3    4  221  212   27   11    2   64    7
  300    3  924    3  353    3  576    3  307    3    4   24    6    9
   29   80    4   84    5    9   69    2  153   17    5  134    4   70
   10    7  924   79   85    3   19  126   12    3 1135   28    7  924
    2   34   92  316  601    5  162  124    3   18   39  151  542    2
   35   71    4  206    7  307    2  120    4   40    7   31    5    9
   21   22   67   60    2   15    2  255    6    7  265    3  428    3
  109  104    3    4  252   37    2   63    8    7   33    2   36    2
    5 1567    9  212    3   64   11 2863  105   28    9  370  387  207
    2  166  256   14    7 1733   31   28    7  613 1908  215    3  447
    9   11   13   53  813    2   72    8  688  160   14  221    3   46
  256   14    7  711    2  263   99    7 2863    3  284  446    3    4
   68    2   92   87  704    3  166    7  924    6    7  168    3   72
    8    7  221    4  711    3    4  255   99    7  169    3 1286   96
    3 

In [40]:
# 레시피와 한 단어 이동한 동일 텍스트로 훈련 세트를 만듭니다.
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

train_ds = text_ds.map(prepare_inputs)

In [41]:
# lstm model
import keras

inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(10000, 100)(inputs)
x = layers.LSTM(128, return_sequences=True)(x)
outputs = layers.Dense(10000, activation="softmax")(x)
lstm = keras.models.Model(inputs, outputs)
lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, None, 128)         117248    
                                                                 
 dense (Dense)               (None, None, 10000)       1290000   
                                                                 
Total params: 2,407,248
Trainable params: 2,407,248
Non-trainable params: 0
_________________________________________________________________


In [43]:
# train lstm

loss_fn = keras.losses.SparseCategoricalCrossentropy()
lstm.compile(optimizer='adam', loss= loss_fn)
lstm.fit(train_ds, epochs=25)

Epoch 1/25


2024-04-15 16:28:54.438026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-15 16:28:54.675556: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-15 16:28:58.619724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


629/629 [==============================] - 124s 167ms/step - loss: 4.4619
Epoch 2/25
629/629 [==============================] - 90s 143ms/step - loss: 3.1639
Epoch 3/25
629/629 [==============================] - 92s 146ms/step - loss: 2.5575
Epoch 4/25
629/629 [==============================] - 148s 236ms/step - loss: 2.2858
Epoch 5/25
629/629 [==============================] - 91s 145ms/step - loss: 2.1279
Epoch 6/25
629/629 [==============================] - 95s 151ms/step - loss: 2.0199
Epoch 7/25
629/629 [==============================] - 95s 151ms/step - loss: 1.9409
Epoch 8/25
629/629 [==============================] - 96s 152ms/step - loss: 1.8802
Epoch 9/25
629/629 [==============================] - 96s 153ms/step - loss: 1.8327
Epoch 10/25
629/629 [==============================] - 99s 157ms/step - loss: 1.7941
Epoch 11/25
629/629 [==============================] - 101s 160ms/step - loss: 1.7620
Epoch 12/25
629/629 [==============================] - 103s 164ms/step - loss: 1.7

In [44]:
# 시작 프롬프트 토큰화
# TextGenerator 체크포인트 만들기


import numpy as np

class TextGenerator(keras.callbacks.Callback):
    def __init__(self, index_to_word, model, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }
        self.model = model

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\n생성된 텍스트:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("recipe for", max_tokens=100, temperature=1.0)


text_generator = TextGenerator(vocab, lstm)


In [45]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\n프롬프트: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 /", max_tokens=10, temperature=1.0
)

print_probs(info, vocab)

2024-04-15 17:10:30.720777: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-15 17:10:30.891835: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-15 17:10:32.779290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-15 17:10:32.850253: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



생성된 텍스트:
recipe for roasted vegetables | chop 1 / 2 tablespoon


프롬프트: recipe for roasted vegetables | chop 1 /
2:   	45.17%
4:   	43.06%
3:   	6.43%
8:   	3.22%
5:   	0.36%
--------


프롬프트: recipe for roasted vegetables | chop 1 / 2
cup:   	40.65%
teaspoon:   	15.89%
tablespoon:   	6.17%
-:   	4.99%
of:   	4.46%
--------

